In [169]:
import pandas as pd
import numpy as np
from scipy.stats import lognorm
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

In [170]:
in_file_name = 'data/grouped_paces_ju.tsv'
df_all = pd.read_csv(in_file_name, delimiter="\t")
history = df_all

In [171]:
order18 = pd.read_csv('data/running_order_j2018_ju.tsv', delimiter="\t")


In [172]:
#paces = df.as_matrix(["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"])
paces = history[["pace_1", "pace_2", "pace_3", "pace_4", "pace_5", "pace_6"]]
logs = np.log(paces)
means = np.nanmean(logs, axis=1)
stdevs = np.nanstd(logs, axis=1)
history = history.assign(log_means=pd.Series(means).values)
history = history.assign(log_stdevs=pd.Series(stdevs).values)

/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.
/Users/oskari/.virtualenvs/jukola-xml-model-PGRmw6ze/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [173]:
# Estimate values for all, but only use them if not other value is available
with_history = history[np.isfinite(df_all.pace_2)]
x = with_history.mean_team_id.values
x = x.reshape(len(x), 1)

log_means = with_history.log_means.values.reshape(len(with_history.log_means), 1)
log_means_model = linear_model.LinearRegression()
log_means_model.fit(x, log_means)

estimated_log_means = log_means_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_means=estimated_log_means)

log_stdevs = with_history.log_stdevs.values.reshape(len(with_history.log_stdevs), 1)
log_stdevs_model = linear_model.LinearRegression()
log_stdevs_model.fit(x, log_stdevs)

estimated_log_stdevs = log_stdevs_model.predict(order18.team_id.values.reshape(len(order18.team_id), 1))
order18 = order18.assign(estimated_log_stdevs=estimated_log_stdevs)
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs
0,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,1,11.0,-,1.944439,0.084991
1,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,2,11.9,-,1.944439,0.084991
2,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,3,12.8,-,1.944439,0.084991
3,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,4,8.7,-,1.944439,0.084991
4,4,Södertälje Nykvarn Orientering 1,Södertälje Nykvarn Orientering,5,8.7,-,1.944439,0.084991


In [174]:
# Combine history with 2018 runners 
no_history_row = pd.DataFrame([[0,0]], columns=["log_means", "log_stdevs"])
def get_history_row(running_order_row):
    name = running_order_row["name"].lower()
    
    by_name = history[history['name'] == name]
    by_name_and_colon = history[history['name'].str.contains(name + ":", regex=False)]

    runners = by_name.append(by_name_and_colon)
    if(len(runners) == 1):
        return runners
    team_name = running_order_row["team_base_name"].upper()
    runners = runners[runners['teams'].str.contains(team_name, regex=False)]
    if(len(runners) == 1):
        return runners
    if(len(runners) == 0):
        return no_history_row
    print(f"name '{name}' team_name '{team_name}'")
    print(f"by_name {len(by_name)} by_name_and_colon {len(by_name_and_colon)} runners {len(runners)}")
    print(f"Duplicate runner {runners}")
    #print(f"TEAMS by_name_and_colon {by_name_and_colon['teams']}")
    return runners.sort_values("num_runs", ascending = False).head(1)

def get_estimate_params(running_order_row):
    history_row = get_history_row(running_order_row)
    #print(f"estimate_row log_means {history_row.log_means} {history_row.log_stdevs}")
    log_means = history_row.log_means.values[0]
    log_stdevs = history_row.log_stdevs.values[0]
    return pd.Series({"history_log_means": log_means, "history_log_stdevs": log_stdevs})

order18 = order18[order18['team'].str.contains("Reak")]
estimate_params = order18.apply(lambda row: get_estimate_params(row), axis=1)
order18 = order18.assign(history_log_means = estimate_params.history_log_means)
order18 = order18.assign(history_log_stdevs = estimate_params.history_log_stdevs)

In [175]:
order18['log_means'] = np.where(np.isfinite(order18["history_log_means"]) & order18["history_log_means"] > 0, order18["history_log_means"], order18["estimated_log_means"])
order18['log_stdevs'] = np.where(np.isfinite(order18["history_log_stdevs"]) & order18["history_log_stdevs"] > 0, order18["history_log_stdevs"], order18["estimated_log_stdevs"])
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs
928,429,Reaktor Innovations 1,Reaktor Innovations,1,11.0,Mikko Peltonen,2.128667,0.092121,2.184951,0.081299,2.184951,0.081299
929,429,Reaktor Innovations 1,Reaktor Innovations,2,11.9,Pasi Huhtiniemi,2.128667,0.092121,2.222239,0.072482,2.222239,0.072482
930,429,Reaktor Innovations 1,Reaktor Innovations,3,12.8,Joni Freeman,2.128667,0.092121,2.130879,0.113846,2.130879,0.113846
931,429,Reaktor Innovations 1,Reaktor Innovations,4,8.7,Tuomas Kareinen,2.128667,0.092121,2.293240,0.160070,2.293240,0.160070
932,429,Reaktor Innovations 1,Reaktor Innovations,5,8.7,Janne Vaittinen,2.128667,0.092121,2.324384,0.039861,2.324384,0.039861


In [176]:
# Calculate personal estimates
# s = sigma and scale = exp(mu).

log_means = np.exp(order18['log_means']) 
log_stdevs = order18['log_stdevs']

intervals95 = lognorm.interval(0.95, s = log_stdevs, scale = log_means)
means = lognorm.mean(s = log_stdevs, scale = log_means)
medians = lognorm.median(s = log_stdevs, scale = log_means)

In [177]:
order18 = order18.assign(ind_95_start = pd.Series(intervals95[0] * order18.leg_dist).values)
order18 = order18.assign(ind_95_end = pd.Series(intervals95[1] * order18.leg_dist).values)
order18 = order18.assign(ind_mean = pd.Series(means * order18.leg_dist).values)
order18 = order18.assign(ind_median = pd.Series(medians * order18.leg_dist).values)

In [178]:
order18.head()

,team_id,team,team_base_name,leg,leg_dist,name,estimated_log_means,estimated_log_stdevs,history_log_means,history_log_stdevs,log_means,log_stdevs,ind_95_start,ind_95_end,ind_mean,ind_median
928,429,Reaktor Innovations 1,Reaktor Innovations,1,11.0,Mikko Peltonen,2.128667,0.092121,2.184951,0.081299,2.184951,0.081299,83.388003,114.684964,98.116094,97.792383
929,429,Reaktor Innovations 1,Reaktor Innovations,2,11.9,Pasi Huhtiniemi,2.128667,0.092121,2.222239,0.072482,2.222239,0.072482,95.270067,126.575551,110.101683,109.812846
930,429,Reaktor Innovations 1,Reaktor Innovations,3,12.8,Joni Freeman,2.128667,0.092121,2.130879,0.113846,2.130879,0.113846,86.244880,134.754922,108.505909,107.805019
931,429,Reaktor Innovations 1,Reaktor Innovations,4,8.7,Tuomas Kareinen,2.128667,0.092121,2.293240,0.160070,2.293240,0.160070,62.980977,117.953879,87.302084,86.190780
932,429,Reaktor Innovations 1,Reaktor Innovations,5,8.7,Janne Vaittinen,2.128667,0.092121,2.324384,0.039861,2.324384,0.039861,82.235006,96.142589,88.987969,88.917301


In [179]:
by_teams = order18.set_index(["team_id", "leg"]).unstack()
by_teams.head()

team                                                \
leg                          1                      2                      3   
team_id                                                                        
429      Reaktor Innovations 1  Reaktor Innovations 1  Reaktor Innovations 1   
1270     Reaktor Innovations 2  Reaktor Innovations 2  Reaktor Innovations 2   

                                                                              \
leg                          4                      5                      6   
team_id                                                                        
429      Reaktor Innovations 1  Reaktor Innovations 1  Reaktor Innovations 1   
1270     Reaktor Innovations 2  Reaktor Innovations 2  Reaktor Innovations 2   

                                     team_base_name                       \
leg                          7                    1                    2   
team_id                                                                    
429      Reaktor Innovations 1  Reaktor Innovations  Reaktor Innovations   
1270     Reaktor Innovations 2  Reaktor Innovations  Reaktor Innovations   

                                 ...       ind_mean                          \
leg                        3     ...              5           6           7   
team_id                          ...                                          
429      Reaktor Innovations     ...      88.987969   91.145398  161.474599   
1270     Reaktor Innovations     ...      98.210347  131.421494  157.389252   

         ind_median                                                            \
leg               1           2           3          4          5           6   
team_id                                                                         
429       97.792383  109.812846  107.805019  86.190780  88.917301   90.759473   
1270     116.864000  133.115223  178.927107  67.174301  96.512277  130.682049   

                     
leg               7  
team_id              
429      161.284847  
1270     156.503700  

[2 rows x 98 columns]

In [180]:
def simulate_relay_estimates(row):
    samples = pd.DataFrame()
    for i in range(1,8):
        samples[i] = row["leg_dist"][i] * lognorm.rvs(s = row["log_stdevs"][i], scale = np.exp(row["log_means"][i]), size = 10000)

    samples_sums = pd.DataFrame()
    for i in range(1,8):
        samples_sums[i] = np.sum([ samples[j] for j in range(1,i+1) ], axis=0)

    start95 = samples_sums.quantile(0.025)
    end95 = samples_sums.quantile(0.975)
    medians = samples_sums.median()
    means = samples_sums.mean()
    
    """
    for i in range(1,8):
        bins = int(samples_sums[i].max() - samples_sums[i].min())
        name = row["name"][i]
        plt.title(f"{name} bins = {bins}")
        plt.hist(samples_sums[i], bins=bins)
        #plt.axvline(x=row["fin_real"][i], color="r")        
        plt.axvline(x=medians[i], color="g")
        plt.axvline(x=means[i], color="yellow")
        plt.show()    
    """
    
    """
    bins = int( (samples_sums.max().max() - samples_sums.min().min()) / 5) 
    plt.figure(figsize=(20, 6))
    plt.title(f"Whole team, bins = {bins}")
    plt.hist([samples_sums[1], samples_sums[2], samples_sums[3], samples_sums[4], samples_sums[5], samples_sums[6], samples_sums[7]], bins=bins)
    for i in range(1,8):
        #plt.axvline(x=row["fin_real"][i], color="r")
        plt.axvline(x=medians[i], color="g")

    plt.show()
    """
    
    fin_start95_dict = {f"fin_start95_{leg}" : start95.values[leg-1] for leg in range(1,8)}
    fin_end95_dict = {f"fin_end95_{leg}" : end95.values[leg-1] for leg in range(1,8)}
    fin_median_dict = {f"fin_median_{leg}" : medians.values[leg-1] for leg in range(1,8)}
    fin_mean_dict = {f"fin_mean_{leg}" : means.values[leg-1] for leg in range(1,8)}
    new_cols = {**fin_start95_dict, **fin_end95_dict, **fin_median_dict, **fin_mean_dict}

    #print(start95.values)
    #print(new_cols)
    return pd.Series(new_cols)

relay_estimates = by_teams.apply(simulate_relay_estimates, axis=1)
relay_estimates.head()

,fin_start95_1,fin_start95_2,fin_start95_3,fin_start95_4,fin_start95_5,fin_start95_6,fin_start95_7,fin_end95_1,fin_end95_2,fin_end95_3,...,fin_median_5,fin_median_6,fin_median_7,fin_mean_1,fin_mean_2,fin_mean_3,fin_mean_4,fin_mean_5,fin_mean_6,fin_mean_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,83.435363,186.871562,285.911184,363.584167,451.757217,540.312902,698.478295,114.805405,231.320981,350.844054,...,492.320215,583.545878,745.202367,98.211508,208.213069,316.738809,404.163566,493.182160,584.415930,745.799897
1270,94.763603,219.853661,364.528033,431.088144,519.722200,646.233080,795.959476,144.027264,285.504471,518.652508,...,595.977408,727.975742,885.669982,117.649822,251.433710,433.368768,500.721151,598.879977,730.386491,887.648918


In [181]:
# Flatten the troublesome multi-index to field_{leg} etc...
by_teams_flat = by_teams.copy()
by_teams_flat.columns = [f'{x[0]}_{x[1]}' for x in by_teams_flat.columns]
by_teams_flat.reset_index()
by_teams_flat

,team_1,team_2,team_3,team_4,team_5,team_6,team_7,team_base_name_1,team_base_name_2,team_base_name_3,...,ind_mean_5,ind_mean_6,ind_mean_7,ind_median_1,ind_median_2,ind_median_3,ind_median_4,ind_median_5,ind_median_6,ind_median_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations 1,Reaktor Innovations,Reaktor Innovations,Reaktor Innovations,...,88.987969,91.145398,161.474599,97.792383,109.812846,107.805019,86.190780,88.917301,90.759473,161.284847
1270,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations 2,Reaktor Innovations,Reaktor Innovations,Reaktor Innovations,...,98.210347,131.421494,157.389252,116.864000,133.115223,178.927107,67.174301,96.512277,130.682049,156.503700


In [182]:
estimates = pd.concat([by_teams_flat, relay_estimates], axis=1, join='inner')

In [183]:
# Convert minutes to date and times
start_timestamp = pd.Timestamp(year = 2018, month = 6, day = 16, hour = 23)

for leg in range(1,8):
    estimates[f"fint_median_{leg}"] = pd.to_datetime(estimates[f"fin_median_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_start95_{leg}"] = pd.to_datetime(estimates[f"fin_start95_{leg}"] * 60, unit = "s", origin= start_timestamp)
    estimates[f"fint_end95_{leg}"] = pd.to_datetime(estimates[f"fin_end95_{leg}"] * 60, unit = "s", origin= start_timestamp)

estimates["fint_median_1"]  

team_id
429    2018-06-17 00:37:51.745829
1270   2018-06-17 00:57:04.320463
Name: fint_median_1, dtype: datetime64[ns]

In [184]:
estimates.to_csv('data/team_estimates_ju2018.tsv', sep="\t")

In [185]:

estimates[[f"{field_prefix}_{leg}" for leg in range(1,8) for field_prefix in ["name", "fint_median", "ind_median"]]]    

,name_1,fint_median_1,ind_median_1,name_2,fint_median_2,ind_median_2,name_3,fint_median_3,ind_median_3,name_4,...,ind_median_4,name_5,fint_median_5,ind_median_5,name_6,fint_median_6,ind_median_6,name_7,fint_median_7,ind_median_7
team_id,,,,,,,,,,,,,,,,,,,,,
429,Mikko Peltonen,2018-06-17 00:37:51.745829,97.792383,Pasi Huhtiniemi,2018-06-17 02:27:48.512522999,109.812846,Joni Freeman,2018-06-17 04:16:21.937397,107.805019,Tuomas Kareinen,...,86.190780,Janne Vaittinen,2018-06-17 07:12:19.212922,88.917301,Olavi Kanerva,2018-06-17 08:43:32.752684,90.759473,Jyri Kytömäki,2018-06-17 11:25:12.142035,161.284847
1270,Antti-Ville Jokela,2018-06-17 00:57:04.320463,116.864000,Karri-Pekka Laakso,2018-06-17 03:10:41.773805000,133.115223,Oskari Pirttikoski,2018-06-17 06:10:21.881996,178.927107,Jeppe Koivula,...,67.174301,Mikko Apo,2018-06-17 08:55:58.644474,96.512277,Tommi Simell,2018-06-17 11:07:58.544529,130.682049,Aki Kaivola,2018-06-17 13:45:40.198904,156.503700


In [190]:
first_names = list(filter(lambda c: "_1" in c,estimates.columns.values))
column_base_names = list(map(lambda c: c[:-2], first_names))
runner_estimates = pd.wide_to_long(estimates.reset_index(), stubnames=column_base_names, i ="team_id", j="leg", sep = "_").sort_values(by=['team_id', 'leg'])
runner_estimates = runner_estimates.drop(['team_base_name', 'estimated_log_means', 'estimated_log_stdevs'], axis=1)
#column_base_names
runner_estimates.head()

team  leg_dist             name  \
team_id leg                                                     
429     1    Reaktor Innovations 1      11.0   Mikko Peltonen   
        2    Reaktor Innovations 1      11.9  Pasi Huhtiniemi   
        3    Reaktor Innovations 1      12.8     Joni Freeman   
        4    Reaktor Innovations 1       8.7  Tuomas Kareinen   
        5    Reaktor Innovations 1       8.7  Janne Vaittinen   

             history_log_means  history_log_stdevs  log_means  log_stdevs  \
team_id leg                                                                 
429     1             2.184951            0.081299   2.184951    0.081299   
        2             2.222239            0.072482   2.222239    0.072482   
        3             2.130879            0.113846   2.130879    0.113846   
        4             2.293240            0.160070   2.293240    0.160070   
        5             2.324384            0.039861   2.324384    0.039861   

             ind_95_start  ind_95_end    ind_mean  ind_median  fin_start95  \
team_id leg                                                                  
429     1       83.388003  114.684964   98.116094   97.792383    83.435363   
        2       95.270067  126.575551  110.101683  109.812846   186.871562   
        3       86.244880  134.754922  108.505909  107.805019   285.911184   
        4       62.980977  117.953879   87.302084   86.190780   363.584167   
        5       82.235006   96.142589   88.987969   88.917301   451.757217   

              fin_end95  fin_median    fin_mean                   fint_median  \
team_id leg                                                                     
429     1    114.805405   97.862430   98.211508 2018-06-17 00:37:51.745829000   
        2    231.320981  207.808542  208.213069 2018-06-17 02:27:48.512522999   
        3    350.844054  316.365623  316.738809 2018-06-17 04:16:21.937397000   
        4    449.860366  403.339921  404.163566 2018-06-17 05:43:20.395278000   
        5    539.601995  492.320215  493.182160 2018-06-17 07:12:19.212922000   

                          fint_start95                 fint_end95  
team_id leg                                                        
429     1   2018-06-17 00:23:26.121751 2018-06-17 00:54:48.324274  
        2   2018-06-17 02:06:52.293747 2018-06-17 02:51:19.258867  
        3   2018-06-17 03:45:54.671050 2018-06-17 04:50:50.643227  
        4   2018-06-17 05:03:35.049993 2018-06-17 06:29:51.621980  
        5   2018-06-17 06:31:45.433023 2018-06-17 07:59:36.119707

In [189]:
runner_estimates.to_csv('data/runner_estimates_ju2018.tsv', sep="\t")